<a href="https://colab.research.google.com/github/ceciliasecchi/MasterThesis/blob/main/MasterThesis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install wordfreq
from wordfreq import word_frequency
from nltk.collections import OrderedDict
import gensim
import pandas as pd
import nltk
nltk.download('punkt')
import re  # For preprocessing
import string
import numpy as np
from numpy import linalg as LA
import csv
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
from gensim.parsing.preprocessing import remove_stopwords,preprocess_string,preprocess_documents
warnings.filterwarnings(action = 'ignore')
import gensim
from gensim.models import Word2Vec
import collections
import math
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import bernoulli 
from scipy.special import softmax as scipy_softmax


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
stopp = {'and','but','this','said','you','the','how','for','like','thats','hes',
         'like','thought','again','but','dont','that','ive','didnt','they','she',
         'while','instead','would','thats','anything','its','hasnt','still',
         'gets','went',"wouldnt","get","let",'way',"these","those","havent"
         'looked','came','not','got','then','ive',"were",'there','wont'}

# Reads 'tweets.txt’ file
sample = open("tweets.txt", "r")
s = sample.read()

voc = {}
vocGlove = OrderedDict()
vocGloveReverse = {}
with open('vocGlove_tweets.csv', mode='r') as inp:
    reader = csv.reader(inp)
    for row in reader:
      voc = {rows[0]:rows[1].replace("\n", " ").replace("["," ").replace("]"," ") for rows in reader}
    
for key,value in voc.items():
  value=value.split()
  if len(value)!=50:
    print(key)
  if key not in stopp:
    vocGlove[key]=np.array(list(map(float,value)))

# normalize the vectors
for k,v in vocGlove.items():
  v = v/np.linalg.norm(v)
  vocGlove[k] = v
  vocGloveReverse[str(v)]=k


In [30]:
from collections import defaultdict


dizionario=defaultdict(int)
# Replaces escape character with space

f = s.split("\n")
data = []
# iterate through each sentence in the file
frasi=[]
for i in f:
  copia=i
  i = remove_stopwords(i)
  i = i.translate(i.maketrans('', '', string.punctuation+"\—\-\"\'\“\”\’\‘"))
  temp = []
	
	# tokenize the sentence into words
  for j in word_tokenize(i):
    j = j.lower()
    if j in vocGlove.keys() and len(j)>2:
      dizionario[j] += 1
      temp.append(j)
    else: pass
  if len(temp)>5:
    data.append(temp)
    frasi.append(copia)
from typing import DefaultDict
def generate_dictionary_data(text):
    word_to_index= OrderedDict()
    index_to_word = OrderedDict()
    freq_dict = DefaultDict(int)
    corpus = []
    corpus_sent = []
    count = 0
    vocab_size = 0
    
    for row in text:
        for word in row:
            freq_dict[word] += 1
            word = word.lower()
            corpus.append(word)
            if word_to_index.get(word) == None:
                word_to_index.update ( {word : count})
                index_to_word.update ( {count : word })
                count  += 1
    vocab_size = len(word_to_index)
    length_of_corpus = len(corpus)
    
    return word_to_index,index_to_word,corpus,vocab_size,length_of_corpus,freq_dict

In [31]:
word_to_index,index_to_word,corpus,vocab_size,length_of_corpus,freq_dict=generate_dictionary_data(data)

In [32]:
sentencesAsVec=[]
num_sent=len(data)

count=0
conteggio={}
for i in range(num_sent):
  len_sent=len(data[i])
  vettori=[]
  for parola in corpus[count:count+len_sent]:
    vettori.append(vocGlove[parola])
  conteggio[i]=[count,count+len_sent]
  count += len_sent
  sentencesAsVec.append(vettori)


# Kernel distances

In [33]:
K=3 #number of Dirac masses in the compressed distribution
N=50 #ambient dimension
J=1000 #how many samples are included: more the better, otherwise there are oscillations in the SGD/Adam convergence
quant=tf.Variable(np.random.rand(N,K),dtype=tf.float32) #this will be the set of quantization variables


In [34]:
def empirical_sampling_from_sentence(sentence, S=30):
    ''' sample from the sentence (that is a vector of words of length 50) 30 words
        note: sentence is a vector of vectors, then the output is a np.array 
              where each coloumn corresponds to a word
    '''
    N=len(sentence)
    Y=[]
    if N>=S:
        indices=np.random.choice(N, S)
    else:
        indices=np.concatenate((np.arange(N),np.random.choice(N,S-N)),axis=0)
        #Y=sentence
    for i in indices:
        Y.append(sentence[i])
    Y=np.transpose(np.asarray(Y))
    return Y
    #return Y,indices

empirical_sampling=empirical_sampling_from_sentence

def empirical_radon_sobolev_distance_sq(X,Y,local_alphas=None,betas=None):
    '''
    Parameters
    ----------
    X : 2D NxK matrix
        input data sample, each column a vector of dimension N, notation X_k
    Y : same as X for the second distribution
    alphas : 1D array of weights for X, in my case alphas==1/K
    betas : same as alphas for Y , in my case betas==1/30?

    Note: X,alphas are Tensor/Variable while Y/betas are numpy array

    Returns
    -------
    Radon-Sobolev distance
    
    '''
    N,K=X.shape
    Ny,J=Y.shape
    Ytensor = tf.convert_to_tensor(Y, dtype=tf.float32)

    # this is my case.. maybe in the future put tf-idf as weights?
    if local_alphas is None:
      alphas = tf.convert_to_tensor(np.ones(K)/K,dtype=tf.float32)
    else:
      alphas=tf.nn.softmax(local_alphas)
    if betas is None:
        betas = np.ones(J)/J
    betastensor = tf.convert_to_tensor(betas, dtype=tf.float32)

    # serie di check per vedere che gli input abbiano senso in dimensione
    # tf.rank = # indeces required to uniquely select an element
    assert (tf.rank(X).numpy()==2) & (Y.ndim==2) & (tf.rank(alphas).numpy()==1) & (betas.ndim==1),"invalid input dimensions"
    Ka,=alphas.shape
    Jb,=betas.shape
    assert (N==Ny) & (K==Ka) & (J==Jb), 'invalid input dimensions'

    #construct big matrix: [X,Y]=[X_1,...,X_3,Y_1,...,Y_30]
    points=tf.concat([X,Ytensor],axis=1)
    # construct big vector of weights 
    gammas=tf.concat([alphas,-betastensor],axis=0)

    #construct the distances matrix as a K+J x K+J matrix
    # h(x)=sqrt(a^2+|x|^2)-a, a=10^-5
    distZZ = tf.math.sqrt(1.0e-10+tf.math.reduce_sum(tf.square(tf.expand_dims(points,2)-tf.expand_dims(points,1)),axis=0))-1.0e-5
    # Given a tensor input, this operation returns a tensor of the same type with all dimensions of size 1 removed.
    # I return thee matrix multiplied by the vectors
    return tf.squeeze(-0.5*tf.expand_dims(gammas,0)@distZZ@tf.expand_dims(gammas,1))

def empirical_radon_sobolev_distance_sq_geod(X,Y,local_alphas=None,betas=None):
    '''
    Parameters
    ----------
    X : 2D NxK matrix
        input data sample, each column a vector of dimension N, notation X_k
    Y : same as X for the second distribution
    alphas : 1D array of weights for X, in my case alphas==1/K
    betas : same as alphas for Y , in my case betas==1/30?

    Note: X,alphas are Tensor/Variable while Y/betas are numpy array

    Returns
    -------
    Radon-Sobolev distance
    
    '''
    N,K=X.shape
    Ny,J=Y.shape
    Ytensor = tf.convert_to_tensor(Y, dtype=tf.float32)
    norma=tf.norm(X,axis=0)
    X=X/tf.sqrt(norma**2+1.e-6)

    if local_alphas is None:
      alphas = tf.convert_to_tensor(np.ones(K)/K,dtype=tf.float32)
    else:
      alphas=tf.nn.softmax(local_alphas)
    if betas is None:
        betas = np.ones(J)/J
    betastensor = tf.convert_to_tensor(betas, dtype=tf.float32)

    assert (tf.rank(X).numpy()==2) & (Y.ndim==2) & (tf.rank(alphas).numpy()==1) & (betas.ndim==1),"invalid input dimensions"
    Ka,=alphas.shape
    Jb,=betas.shape
    assert (N==Ny)& (K==Ka)&(J==Jb), 'invalid input dimensions'


    #construct big matrix: [X,Y]=[X_1,...,X_3,Y_1,...,Y_30]
    points=tf.concat([X,Ytensor],axis=1)

    # construct big vector of weights 
    gammas=tf.concat([alphas,-betastensor],axis=0)

    #construct the distances matrix as a K+J x K+J matrix
    
    distZZ = tf.math.acos(tf.matmul(tf.transpose(points),points)/1.0001)
    # Given a tensor input, this operation returns a tensor of the same type with all dimensions of size 1 removed.
    # I return thee matrix multiplied by the vectors
    return tf.squeeze(-0.5*tf.expand_dims(gammas,0)@distZZ@tf.expand_dims(gammas,1))#-sum(np.multiply(Lag,np.power(np.linalg.norm(X,axis=0),2)))+K

                            

In [35]:
def Projection(vector):
  vicini=[]
  lista=[]
  chiavi=list(word_to_index.keys())

  for word in chiavi:
    pr = np.dot(vector, vocGlove[word])
    lista.append(pr)
  maxs=[]
  for i in range(num):
    mm=max(lista)
    mm_index=lista.index(mm)
    maxs.append(mm_index)
    lista.remove(mm)
  for i in maxs:
    vicini.append(chiavi[i])
  return vicini  

def Projection2(vector):
    #vector=np.array([vector]).reshape(-1,50)
    #vector=[vector]
    sentence=[vector]
    N=len(sentence)
    #indici=random.sample(range(N), 3)
    score1={}
    for parola in vocGlove.keys():
      parola_vec=vocGlove[parola]
      compressed=tf.Variable(np.array([parola_vec]).reshape(50,-1), dtype=tf.float32)
      score1[parola]=math.sqrt(distanzaH(sentence,compressed).numpy())
    best1=min(score1.values())
    migliore1=[key for key, value in score1.items() if value == best1]
    parola1=vocGlove[migliore1[0]]
    return parola1,migliore1[0],best1

In [36]:
def distanzaH(sentence,compressed):
        J=len(sentence)

        sentence=np.transpose(sentence)
        N,K=compressed.shape
        Ytensor = tf.convert_to_tensor(sentence, dtype=tf.float32)
        alphas = tf.convert_to_tensor(np.ones(K)/K,dtype=tf.float32)
        betas = np.ones(J)/J
        betastensor = tf.convert_to_tensor(betas, dtype=tf.float32)
        points=tf.concat([compressed,sentence],axis=1)
        gammas=tf.concat([alphas,-betastensor],axis=0)
        distZZ = tf.math.sqrt(1.0e-10+tf.math.reduce_sum(tf.square(tf.expand_dims(points,2)-tf.expand_dims(points,1)),axis=0))-1.0e-5
        return tf.squeeze(-0.5*tf.expand_dims(gammas,0)@distZZ@tf.expand_dims(gammas,1))

def distanzaGeo(sentence,compressed):
        J=len(sentence)
        norma=tf.norm(compressed,axis=0)
        compressed=compressed/tf.sqrt(norma**2+1.e-6)
        sentence=np.transpose(sentence)
        N,K=compressed.shape
        Ytensor = tf.convert_to_tensor(sentence, dtype=tf.float32)
        alphas = tf.convert_to_tensor(np.ones(K)/K,dtype=tf.float32)
        betas = np.ones(J)/J
        betastensor = tf.convert_to_tensor(betas, dtype=tf.float32)
        points=tf.concat([compressed,sentence],axis=1)
        gammas=tf.concat([alphas,-betastensor],axis=0)
        distZZ = tf.math.acos(tf.matmul(tf.transpose(points),points)/1.0001)
        return tf.squeeze(-0.5*tf.expand_dims(gammas,0)@distZZ@tf.expand_dims(gammas,1))



#Doing some statistics
get the distance from compression to the final compression
*   S = sentence (mean)
*   C1 = compression (raw)
*   CP = projected compression
*   CR = random compression (mean)

I will evaluate d(S,CR), d(S,C1), d(S,CP), d(C1,CP)

# First Method
Given the 3 compression vectors, the associated word is found through the cosine distance. Statistics are reported.

In [37]:
for i in range(20,40):
    sentence=sentencesAsVec[i]
    N=len(sentence)

    opt = tf.keras.optimizers.Adam(learning_rate=0.1,beta_1=0.9, beta_2=0.999)# TODO: use lr_schedule for step decay schedule !
    Y=empirical_sampling(sentence)
    #Y,indices=empirical_sampling(sentence)
    loss_quant = lambda: empirical_radon_sobolev_distance_sq(quant,Y)

    nr_iter=100
    loss_val=[]
    opt_val=[]

    for sc in range(nr_iter):
      step_count = opt.minimize(loss_quant, [quant]).numpy()
      opt_val.append(quant.numpy())
      loss_val.append(loss_quant())
      simili=[]
    v1,p1,s1=Projection2(quant[:,0])
    v2,p2,s2=Projection2(quant[:,1])
    v3,p3,s3=Projection2(quant[:,2])
    print(frasi[i])
    print("[",p1.upper(),p2.upper(),p3.upper(),"]")    
    PROJ=tf.Variable(np.array([v1,v2,v3]).reshape(50,-1), dtype=tf.float32)
    indici=np.random.choice(N, 3)
    RANDOM=tf.Variable(np.array([sentence[indici[0]],sentence[indici[1]],sentence[indici[2]]]).reshape(50,-1),dtype=tf.float32)
    CR=tf.Variable(np.random.rand(50,3), dtype=tf.float32)
    print("RS d(S,C1): ",math.sqrt(distanzaH(sentence,quant).numpy()))
    print("RS d(S,CP): ",math.sqrt(distanzaH(sentence,PROJ).numpy()))
    print("RS d(S,CR): ",math.sqrt(distanzaH(sentence,RANDOM).numpy()))


Sorry you don't like being asked to clean up after yourselves, dudes.   Accountability is hard.  Oh, well.
[ YOURSELVES HARD CLEAN ]
RS d(S,C1):  0.28058991856481147
RS d(S,CP):  0.6346816961116057
RS d(S,CR):  0.653727884702632
A major cause for celebration and dancing in the streets: Henry the Hatter to open in Eastern Market on Friday https://t.co/lOaHpbv7xP @crainsdetroit
[ DANCING EASTERN FRIDAY ]
RS d(S,C1):  0.3030915522064531
RS d(S,CP):  0.633876259658275
RS d(S,CR):  0.6525984169924163
my seven-year-old artist daughter has been drawing a lot of cartoon bunnies lately. they've all been extremely cutesey, but today I found this... https://t.co/1jsraU01Nc
[ ARTIST EXTREMELY LOT ]
RS d(S,C1):  0.3019409611510891
RS d(S,CP):  0.6955418369248291
RS d(S,CR):  0.6165946334145038
A couple years ago, a very nice young girl was being bullied for wearing a feminist t-shirt in a photo. I offered her words of support.   Today, in Sao Paulo, she proudly handed me a graphic novel she wrote. 

# Best triplet from the sentence
Searching for the best triplet of words taken from the sentence, using as kernel 
$h(x,y)=\sqrt{|x-y|^2+a^2}-a$.

In [38]:
  for w in range(10,20):
    sentence=sentencesAsVec[w]
    N=len(sentence)
    indici=random.sample(range(N), 3)
    score={}
    for i in range(N):
      for j in range(i+1,N):
        for k in range(j+1,N):
          parole=data[w][i],data[w][j],data[w][k]
          compressed=tf.Variable(np.array([sentence[i],sentence[j],sentence[k]]).reshape(50,-1), dtype=tf.float32)
          score[parole]=math.sqrt(distanzaH(sentence,compressed).numpy())
    best3=sorted(score.values())[:1]
    #migliori=[key for key, value in score.items() if value == best3]
    print(frasi[w])
    migliori=[key for key, value in score.items() if value in best3]
    p1=migliori[0][0].upper()
    p2=migliori[0][1].upper()
    p3=migliori[0][2].upper()
    print("[",p1,p2,p3,"]"," d(S,CT)=",best3)

In retrospect, we should have taken to the streets when McConnell refused to let Obama rightfully fill a Supreme Court vacancy. That was an epic fail on our part as citizens.
[ REFUSED COURT EPIC ]  d(S,CT)= [0.5373318852842112]
Hey, you misguided nincompoops. Here's a glimpse of #FairandBalanced response to a hatemonger https://t.co/inPIz0twO6.
[ HEY NINCOMPOOPS RESPONSE ]  d(S,CT)= [0.5866300686028567]
Social Security and Medicare are not "Entitlements" that can be renegotiated.   The term is "DEFERRED PAY."  I've paid into the system my entire life, and so have you. Anyone stealing your deferred pay needs to be voted out.
[ ENTITLEMENTS PAID ANYONE ]  d(S,CT)= [0.5718858259249879]
'You let a weird guy whisper in your ear and you didn't trust your daughter.' --my 7-year-old girl https://t.co/Iqhx8PUGSo
[ WEIRD GUY GIRL ]  d(S,CT)= [0.606021407378753]
@saladinahmed May the initiative to require neutral redistricting in Michigan succeed.   For those unaware of it: https://t.co/DfBydPOn

using as kernel 
$h(x,y)=\arccos{x\cdot y}$

In [39]:
  for w in range(10,20):
    sentence=sentencesAsVec[w]
    N=len(sentence)
    indici=random.sample(range(N), 3)
    score={}
    for i in range(N):
      for j in range(i+1,N):
        for k in range(j+1,N):
          parole=data[w][i],data[w][j],data[w][k]
          compressed=tf.Variable(np.array([sentence[i],sentence[j],sentence[k]]).reshape(50,-1), dtype=tf.float32)
          score[parole]=math.sqrt(distanzaGeo(sentence,compressed).numpy())
    best3=sorted(score.values())[:1]
    print(frasi[w])
    migliori=[key for key, value in score.items() if value in best3]
    p1=migliori[0][0].upper()
    p2=migliori[0][1].upper()
    p3=migliori[0][2].upper()
    print("[",p1,p2,p3,"]"," d(S,CT)=",best3)


In retrospect, we should have taken to the streets when McConnell refused to let Obama rightfully fill a Supreme Court vacancy. That was an epic fail on our part as citizens.
[ REFUSED COURT EPIC ]  d(S,CT)= [0.5544603715551787]
Hey, you misguided nincompoops. Here's a glimpse of #FairandBalanced response to a hatemonger https://t.co/inPIz0twO6.
[ HEY NINCOMPOOPS RESPONSE ]  d(S,CT)= [0.6147604676278882]
Social Security and Medicare are not "Entitlements" that can be renegotiated.   The term is "DEFERRED PAY."  I've paid into the system my entire life, and so have you. Anyone stealing your deferred pay needs to be voted out.
[ ENTITLEMENTS PAID ANYONE ]  d(S,CT)= [0.5906362451764492]
'You let a weird guy whisper in your ear and you didn't trust your daughter.' --my 7-year-old girl https://t.co/Iqhx8PUGSo
[ WEIRD GUY GIRL ]  d(S,CT)= [0.6341758922936803]
@saladinahmed May the initiative to require neutral redistricting in Michigan succeed.   For those unaware of it: https://t.co/DfBydPO